# Week02 - Basic Prompt Engineering Techniques

본 노트북은 기본적인 프롬프트 엔지니어링 기법들을 실습합니다.
- Ollama (무료)
- OpenAI gpt-5-mini or GPT-4o-mini (유료)

각 기법에 대해 설명과 함께 실제 코드 예시를 제공합니다.

In [10]:
# 필요한 라이브러리 설치 및 import
import subprocess
import json
import re
import time
import random
import os
from dotenv import load_dotenv
from typing import Dict, List, Optional

# .env 파일 로드
load_dotenv()

# OpenAI 라이브러리 설치 (필요시)
try:
    from openai import OpenAI
except ImportError:
    !pip install openai
    from openai import OpenAI

In [ ]:
# 설정
# OpenAI API 키를 환경변수에서 가져오거나 직접 입력
# client = OpenAI(api_key="your-api-key-here")  # 직접 입력시
client = OpenAI()  # 환경변수 사용시

# Ollama 함수 헬퍼
def run_ollama(model: str, prompt: str) -> str:
    """Ollama 모델 실행"""
    try:
        result = subprocess.run(
            ["ollama", "run", model],
            input=prompt,
            text=True,
            capture_output=True,
            timeout=60
        )
        return result.stdout.strip()
    except subprocess.TimeoutExpired:
        return "Error: Timeout"
    except Exception as e:
        return f"Error: {str(e)}"

# OpenAI 함수 헬퍼
def run_openai(prompt: str, model: str = "gpt-5-mini", **kwargs) -> str:
    """OpenAI 모델 실행"""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            **kwargs
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

## 1. 문제 정의 (Problem Definition)

**목표:** 무엇을, 얼마나 잘, 어떤 데이터로 할지 합의

- **태스크 명시:** 요약/분류/추론/계획/코드생성 등
- **입력/출력 규격:** 필수 필드, 포맷(JSON/표/문장)
- **성공기준(정량/정성):** 예) F1≥0.85, 길이≤120자, 금칙어 없음
- **제약:** 시간(p95 응답 ≤ 3s), 비용(≤ $X/1k), 컨텍스트(≤ 32k)
- **평가 데이터:** 골든셋(샘플 N개), 오류 유형(Tag) 정의
- **수용 기준(AC):** "이 조건 충족 시 Pass"를 문장으로 명시

In [ ]:
# 1. 문제 정의 - 고객 리뷰 감성 분류
problem_definition_prompt = """Task: Classify sentiment of the given review.
Output: JSON with keys {label: "positive|negative|neutral", reason: string} in Korean.
Constraints: reason ≤ 20 tokens, no extra text.
Acceptance: JSON parseable, label ∈ {positive,negative,neutral}.
Review: --- 배송도 빠르고 포장도 깔끔했어요. 품질도 아주 좋아요! ---"""

print("=== Ollama (llama3.1:8b) ===")
ollama_result = run_ollama("llama3.1:8b", problem_definition_prompt)
print(ollama_result)

print("\n=== OpenAI (GPT-5-mini) ===")
openai_result = run_openai(problem_definition_prompt, max_completion_tokens=120)
print(openai_result)

=== Ollama (llama3.1:8b) ===
```
{
  "label": "positive",
  "reason": "\uad6c\ud558\uace0\uc2dc \ub098\uc57d\uc758 \uc8fc\uc548\uc138\uc694 \uba38\uc7ac\uc744 \uc88b\uc11d \ubb38\uc694."
}
```

Translation:

* `\uad6c\ud558\uace0` means "배송" (delivery)
* `\ub098\uc57d\uc758` means "빠르고" (fast)
* `\uc8fc\uc548\uc138\uc694` means "포장도 깔끔했어요" (packaging was neat)
* `\uba38\uc7ac\uc744` means "품질" (quality)
* `\ucc44\uc11d` means "좋아요" (good)

=== OpenAI (GPT-5-mini) ===



In [17]:
# 1-2. 다른 예시로 테스트
negative_review_prompt = """Task: Classify sentiment of the given review.
Output: JSON with keys {label: "positive|negative|neutral", reason: string} in Korean.
Constraints: reason ≤ 20 tokens, no extra text.
Acceptance: JSON parseable, label ∈ {positive,negative,neutral}.
Review: --- 배송이 너무 늦고 제품도 설명과 달라서 실망했어요. ---"""

print("=== Negative Review Test ===")
print("Ollama:", run_ollama("llama3.1:8b", negative_review_prompt))
print("OpenAI:", run_openai(negative_review_prompt, temperature=0.2, max_completion_tokens=120))

=== Negative Review Test ===
Ollama: Here is the classification result:

```
{
  "label": "negative",
  "reason": "\ud488\ub0b4 \uba38\ub2e8 \uc758\uc6d0\uc694 \ubc88\uc138\uc694 \uc11d\uc18c\uc77c \ucf54\uc7ac\uc9d1"
}
```

Translation: The delivery was too late and the product is different from what was described, so I was disappointed.
OpenAI: Error: Error code: 400 - {'error': {'message': "Unsupported value: 'temperature' does not support 0.2 with this model. Only the default (1) value is supported.", 'type': 'invalid_request_error', 'param': 'temperature', 'code': 'unsupported_value'}}


## 2. 페르소나 (Persona)

**목표:** 일관된 스타일·관점·전문성 부여

- **역할/전문성:** "너는 금융 데이터 애널리스트"
- **관점/목표:** "리스크 최소화와 규제 준수 최우선"
- **금지/선호:** 마케팅 톤 금지, 표/JSON 선호
- **심화:** 대상 독자(경영진/개발자/학생)에 맞춘 수준 조절

In [18]:
# 2. 페르소나 - 중학교 과학 교사
persona_prompt_ollama = """You are a middle-school science teacher.
Explain photosynthesis in Korean in ≤3 sentences, include 1 everyday example, friendly tone."""

print("=== Ollama (중학교 과학 교사 페르소나) ===")
persona_ollama = run_ollama("mistral", persona_prompt_ollama)
print(persona_ollama)

=== Ollama (중학교 과학 교사 페르소나) ===
화학적으로 말해보면, 수선이란 나무와 식물들이 일상생활에서도 많이 보여지는 해빛을 활용하여 CO₂ (탄소다이오кси드)와 H₂O (수소)를 가지고 탄소, 지구의 열과 영양소를 만들어내는 것입니다. 예를 들면, 식물의 잎에서 아름답게 빛나는 초록색은 수선의 작업 결과입니다!

(Translated to English: Chemically speaking, photosynthesis is a process in which trees and plants use sunlight that we see every day, along with CO₂ (carbon dioxide) and H₂O (water), to produce carbon, heat for the Earth, and nutrients. For example, the beautiful green color you see on plant leaves is the result of photosynthesis!)


In [19]:
# OpenAI용 페르소나 (시스템 메시지 활용)
persona_messages = [
    {"role": "system", "content": "You are a middle-school science teacher. Friendly, concise, concrete examples."},
    {"role": "user", "content": "광합성을 3문장 이내로 설명하고 일상 예시 1개를 들어줘."}
]

print("=== OpenAI (시스템 메시지 활용) ===")
persona_openai = client.chat.completions.create(
    model="gpt-5-mini",
    messages=persona_messages,
    max_completion_tokens=150
).choices[0].message.content
print(persona_openai)

=== OpenAI (시스템 메시지 활용) ===



In [20]:
# 2-2. 다른 페르소나 예시: 금융 분석가
finance_persona = """You are a senior financial analyst specializing in risk assessment.
Analyze this investment scenario in Korean, focus on risk factors, 3 bullets max:
스타트업 A사가 시리즈B 투자 유치를 위해 제안서를 제출했습니다. 
매출은 전년 대비 300% 증가했지만 아직 적자 상태입니다."""

print("=== 금융 분석가 페르소나 비교 ===")
print("Ollama:", run_ollama("llama3.1:8b", finance_persona))
print("\nOpenAI:", run_openai(finance_persona, temperature=0.3, max_tokens=200))

=== 금융 분석가 페르소나 비교 ===
Ollama: 안녕하세요. 한국어 분석을 위한 Risk Assessment 결과는 다음과 같습니다.

* **가치와 가치의 불일치** : 매출이 300% 증가했으나, 여전히 적자인 것을 고려할 때, 현재 투자 비용 대비 이익률은 매우 낮다. 이는 투자 시 충분한 수익성을 보장하지 못하는 위험이 존재한다.
* **수익성 및 경영 능력의 미흡** : 아직 적자 상태인 가운데 매출이 급격하게 증가했다는 사실은 단기적인 성공을 반영할 뿐이다. 투자가 이 벤처의 수익성을 확신할 수 있는지 여부에 대한 불확실성이 존재한다.
* **투자의 부적합성** : 시리즈 B 라운드 투자는 일반적으로 성숙한 스타트업에게 제공되는 최소 10억 달러 이상의 펀딩을 의미한다. 이는 아직 적자 상태인 A사가 적절한 투자자 대상일 수 있는지 여부에 대한 질문을 제기한다.

OpenAI: Error: Error code: 400 - {'error': {'message': "Unsupported parameter: 'max_tokens' is not supported with this model. Use 'max_completion_tokens' instead.", 'type': 'invalid_request_error', 'param': 'max_tokens', 'code': 'unsupported_parameter'}}


## 3. 톤/포맷 (Tone & Format)

**목표:** 재사용/평가 가능한 **일관 포맷**

- **톤:** 공식/친근/단호/코치형 등 지정
- **포맷:** JSON/Markdown 표/불릿/段락 길이
- **스키마:** 키 이름, 자료형, 허용 값(스키마 미스 방지)
- **후처리 용이성:** "JSON만 출력", 코드블록 금지 등

In [21]:
# 3. 톤/포맷 - 뉴스 요약 JSON 스키마
news_format_prompt = """Summarize the news in JSON only:
{headline:string, who:string[], what:string, when:string, where:string, confidence:number[0,1]}.
Text: --- 애플이 내달 초 신형 아이폰 공개를 예고했다. 팀 쿡 CEO는 캘리포니아 쿠퍼티노 본사에서 열린 기자회견에서 
"혁신적인 기능들이 포함될 것"이라고 발표했다. 정확한 출시일은 9월 15일로 예상된다고 업계 관계자들이 전했다. ---"""

print("=== 뉴스 요약 JSON 포맷 ===")
print("Ollama:")
news_ollama = run_ollama("llama3.1:8b", news_format_prompt)
print(news_ollama)

print("\nOpenAI:")
news_openai = run_openai(news_format_prompt, max_completion_tokens=220)
print(news_openai)

=== 뉴스 요약 JSON 포맷 ===
Ollama:
```
{
  "headline": "애플, 신형 아이폰 공개 예고",
  "who": ["팀 쿡 CEO"],
  "what": "혁신적인 기능 포함",
  "when": "내달 초 (예상)",
  "where": "캘리포니아 쿠퍼티노",
  "confidence": 0.8
}
```

OpenAI:



In [22]:
# JSON 파싱 테스트
def test_json_parsing(text: str) -> bool:
    """JSON 파싱 가능 여부 테스트"""
    try:
        # 코드 블록 제거
        clean_text = re.sub(r'```json\s*|```\s*', '', text).strip()
        json.loads(clean_text)
        return True
    except:
        return False

print("=== JSON 파싱 테스트 결과 ===")
print(f"Ollama JSON 파싱 가능: {test_json_parsing(news_ollama)}")
print(f"OpenAI JSON 파싱 가능: {test_json_parsing(news_openai)}")

=== JSON 파싱 테스트 결과 ===
Ollama JSON 파싱 가능: True
OpenAI JSON 파싱 가능: False


## 4. Delimiter/길이 제어

**목표:** 입력 경계/출력 길이로 오류·헛수고 방지

- **명시적 경계:** `<<<INPUT>>> ... <<<END>>>`
- **금지사항:** 경계 바깥 문맥 무시
- **길이제어:** 문장 수, 불릿 개수, 최대 토큰, 글자 수
- **Truncation 대비:** 가장 중요한 정보 먼저 배치

In [23]:
# 4. Delimiter/길이 제어
delimiter_prompt = """Summarize within 3 bullets (≤80 words). Ignore text outside delimiters.
<<<INPUT>>>
이번 제품 업데이트는 성능 향상과 배터리 시간을 대폭 개선했습니다. 
새로운 AI 칩셋을 탑재하여 처리 속도가 40% 향상되었고, 
배터리는 기존 대비 6시간 더 오래 사용할 수 있습니다.
사용자 인터페이스도 더욱 직관적으로 개선되었습니다.
<<<END>>>
이 텍스트는 무시해주세요. 관련 없는 내용입니다."""

print("=== Delimiter & 길이 제어 테스트 ===")
print("Ollama:")
delimiter_ollama = run_ollama("llama3.1:8b", delimiter_prompt)
print(delimiter_ollama)

print("\nOpenAI:")
delimiter_openai = run_openai(delimiter_prompt, max_completion_tokens=140)
print(delimiter_openai)

=== Delimiter & 길이 제어 테스트 ===
Ollama:
* 이번 제품 업데이트로 성능 향상과 배터리 시간 개선이 이루어졌습니다.
* 새로운 AI 칩셋 탑재로 처리 속도가 40% 향상되고, 배터리는 기존 대비 6시간 더 오래 사용할 수 있습니다.
* 사용자 인터페이스도 더욱 직관적으로 개선되었습니다.

OpenAI:



In [24]:
# 길이 제어 검증
def count_bullets_and_words(text: str) -> tuple:
    """불릿 개수와 단어 수 카운트"""
    bullets = text.count('•') + text.count('-') + text.count('*')
    # 간단한 단어 수 계산 (한글은 글자 수로)
    korean_chars = len(re.findall(r'[가-힣]', text))
    english_words = len(re.findall(r'\b[a-zA-Z]+\b', text))
    return bullets, korean_chars + english_words

ollama_bullets, ollama_length = count_bullets_and_words(delimiter_ollama)
openai_bullets, openai_length = count_bullets_and_words(delimiter_openai)

print("=== 길이 제어 검증 ===")
print(f"Ollama - 불릿 수: {ollama_bullets}, 길이: {ollama_length}")
print(f"OpenAI - 불릿 수: {openai_bullets}, 길이: {openai_length}")
print(f"제약 조건: 불릿 3개, 80단어 이내")

=== 길이 제어 검증 ===
Ollama - 불릿 수: 3, 길이: 91
OpenAI - 불릿 수: 0, 길이: 0
제약 조건: 불릿 3개, 80단어 이내


## 5. Pre-warming (대화 히스토리 시드)

**목표:** 대화 초반부터 도메인 맥락/스타일을 "데워" 일관성↑

- **브랜드/용어 통일:** 제품명, 정책, 금칙어
- **샘플 Q/A 포함:** 답변 레벨 가이드
- **고정 규칙:** 포맷·톤, 예외 처리 원칙
- **캐싱 고려:** 자주 쓰는 프리앰블 공유

In [25]:
# 5. Pre-warming - 쇼핑몰 CS봇 히스토리 시드
prewarming_prompt_ollama = """System Memory:
- 브랜드명: PulseFit Pro
- 환불 정책: 수령 14일 이내 미개봉 전액 환불
- 톤: 정중·간결
Q: 반품은 어떻게 하나요?
A: (위 정책에 맞춰 2문장으로 답해줘)"""

print("=== Pre-warming (Ollama) ===")
prewarming_ollama = run_ollama("llama3.1:8b", prewarming_prompt_ollama)
print(prewarming_ollama)

=== Pre-warming (Ollama) ===
반품은 수령 14일 이내에 가능하며, 미개봉의 조건을 지키신다면 전액 환불이 됩니다.


In [26]:
# OpenAI용 Pre-warming (시스템 메시지 + 대화 히스토리)
prewarming_messages = [
    {"role": "system", "content": "Brand: PulseFit Pro. Refund: within 14 days if unopened. Tone: polite & concise. Format: 2 sentences."},
    {"role": "user", "content": "반품은 어떻게 하나요?"}
]

print("=== Pre-warming (OpenAI) ===")
prewarming_openai = client.chat.completions.create(
    model="gpt-5-mini",
    messages=prewarming_messages,
    max_completion_tokens=120
).choices[0].message.content
print(prewarming_openai)

=== Pre-warming (OpenAI) ===



In [27]:
# 5-2. 연속 대화 시뮬레이션 (OpenAI)
conversation_history = [
    {"role": "system", "content": "Brand: PulseFit Pro. Refund: within 14 days if unopened. Tone: polite & concise. Format: 2 sentences."},
    {"role": "user", "content": "반품은 어떻게 하나요?"},
    {"role": "assistant", "content": prewarming_openai},
    {"role": "user", "content": "교환도 같은 조건인가요?"}
]

print("=== 연속 대화 테스트 ===")
followup_response = client.chat.completions.create(
    model="gpt-5-mini",
    messages=conversation_history,
    max_completion_tokens=120
).choices[0].message.content
print(followup_response)

=== 연속 대화 테스트 ===



## 6. Ask for Context (컨텍스트 요구)

**목표:** 정보 부족 시 **질문 먼저** → 품질·오류 방지

- **필수 필드 정의:** 누락 시 질의 (최대 N문항)
- **질문 스타일:** 폐쇄형(예/아니오), 다지선다, 혹은 짧은 서술
- **중간확인:** 답변 전 "이해한 바"를 1줄로 요약
- **타임아웃/재질의:** 2회 이상 미응답 시 기본값·가정 사용

In [28]:
# 6. Ask for Context - 프로젝트 요약 요청
context_prompt = """You are a project assistant.
If any of {deadline, audience, deliverables} is missing, ask up to 3 questions first, then summarize.
User request: '프로젝트 요약 좀 만들어줘.'"""

print("=== Ask for Context (Ollama) ===")
context_ollama = run_ollama("llama3.1:8b", context_prompt)
print(context_ollama)

print("\n=== Ask for Context (OpenAI) ===")
context_openai = run_openai(context_prompt, max_completion_tokens=250)
print(context_openai)

=== Ask for Context (Ollama) ===
Before I can help you create a project summary, I need some more information. Could you please answer the following questions?

1. What is the main objective of this project?
2. Who is the target audience for this project summary?
3. Are there any specific requirements or outcomes that we should highlight in the summary?

Once I have this information, I can help create a concise and effective project summary!

=== Ask for Context (OpenAI) ===



In [29]:
# 6-2. 충분한 정보가 제공된 경우
complete_context_prompt = """You are a project assistant.
If any of {deadline, audience, deliverables} is missing, ask up to 3 questions first, then summarize.
User request: 'AI 챗봇 개발 프로젝트 요약 만들어줘. 
마감일은 다음달 말이고, 경영진 대상이며, 
최종 산출물은 프로토타입과 기술문서야.'"""

print("=== 충분한 컨텍스트 제공시 ===")
print("Ollama:")
complete_ollama = run_ollama("llama3.1:8b", complete_context_prompt)
print(complete_ollama)

print("\nOpenAI:")
complete_openai = run_openai(complete_context_prompt, max_completion_tokens=300)
print(complete_openai)

=== 충분한 컨텍스트 제공시 ===
Ollama:
Based on your request, here's a summary of the project:

**Project Summary:**

* **Deadline:** Next month (end of the month)
* **Audience:** 경영진 (Executive Management team)
* **Deliverables:** 프로토타입 (Prototype) and 기술문서 (Technical Document)

Is this accurate?

OpenAI:



## 종합 실습: 기법 조합

여러 기법을 조합하여 더 강력한 프롬프트를 만들어봅시다.

In [30]:
# 종합 실습: 고급 고객 지원 봇
combined_prompt = """You are a senior customer service representative for TechPro Solutions.
Brand guidelines: Professional, empathetic, solution-focused.
Required fields: {issue_type, urgency_level, customer_info}

Instructions:
1. If any required field is missing, ask clarifying questions (max 2).
2. Classify issue urgency: high|medium|low
3. Provide response in JSON format:
   {"urgency": "high|medium|low", "next_steps": ["..."], "estimated_time": "...", "escalation_needed": boolean}
4. Keep responses under 100 words.
5. Use empathetic tone for high urgency issues.

<<<INPUT>>>
고객 문의: 어제 주문한 노트북이 아직 배송 준비중이라고 나오는데 언제 받을 수 있나요? 급하게 필요합니다.
<<<END>>>"""

print("=== 종합 실습: 고급 CS봇 ===")
print("OpenAI 결과:")
combined_result = run_openai(combined_prompt, max_completion_tokens=300)
print(combined_result)

# JSON 파싱 테스트
print(f"\nJSON 파싱 가능: {test_json_parsing(combined_result)}")

=== 종합 실습: 고급 CS봇 ===
OpenAI 결과:


JSON 파싱 가능: False


## 성능 비교 및 분석

In [ ]:
# 기법별 성능 비교를 위한 테스트 케이스
test_cases = [
    {
        "name": "Basic Prompt",
        "prompt": "리뷰를 긍정/부정/중립으로 분류하세요: 배송이 늦었지만 제품은 좋아요"
    },
    {
        "name": "Problem Definition",
        "prompt": """Task: Classify sentiment.
Output: JSON {label: "positive|negative|neutral", confidence: 0-1}
Review: --- 배송이 늦었지만 제품은 좋아요 ---"""
    },
    {
        "name": "With Persona",
        "prompt": """You are an expert sentiment analyst.
Classify sentiment with confidence score.
Output JSON: {label: "positive|negative|neutral", confidence: 0-1}
Review: --- 배송이 늦었지만 제품은 좋아요 ---"""
    }
]

print("=== 기법별 성능 비교 ===")
for test_case in test_cases:
    print(f"\n--- {test_case['name']} ---")
    result = run_openai(test_case['prompt'], max_completion_tokens=100)
    print(result)
    print(f"JSON 파싱 가능: {test_json_parsing(result)}")

## 정리 및 베스트 프랙티스

### 기법별 주요 포인트

1. **문제 정의**: 명확한 태스크·포맷·제약 명시
2. **페르소나**: 일관된 관점과 전문성 부여
3. **톤/포맷**: 구조화된 출력으로 후처리 용이성 확보
4. **Delimiter**: 입력 경계와 길이 제어로 정확성 향상
5. **Pre-warming**: 도메인 맥락으로 일관성 확보
6. **Ask for Context**: 정보 부족시 능동적 질문으로 품질 향상

### 실무 적용 가이드

- **프로덕션 환경**에서는 여러 기법을 조합하여 사용
- **테스트 케이스**를 통한 정기적인 성능 검증 필요
- **비용과 성능의 균형**을 고려한 기법 선택
- **사용자 피드백**을 통한 지속적인 개선